# 1. Introduction

In this notebook i will try to find good candidates for Measuring accuracy.

### Selection criterias:
1. No points lying on highway
2. All points are in resedential place
3. Entire trip is covered
4. Amount of trips no anomalic

In [1]:
import os
import numpy as np
import pandas as pd
import plotly.express as px
import geopandas as gpd
from scipy.stats import zscore
from multiprocessing import Pool
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
from GPSOdyssey import Vega, Polaris

# 2. Data

* RFID: RFID activations
* GPS: GPS records from Trucks

### 1. RFID

In [2]:
S_RFID_OCTOBER = '/Data/Source/RFID/RFID_october.csv'
rfid = pd.read_csv(S_RFID_OCTOBER)

### 2. Map matching

In [3]:
I_MAP_MATCHING_REPORT = '/Data/Intermediate/MapMatchingReports/MapMatching_october2020.csv'
map_matching_report = pd.read_csv(I_MAP_MATCHING_REPORT)

### 3. GPS data

In [4]:
S_GPS_DIR = '/Data/Source/OctoberGPS/'
gps = Vega().read_from_dir(directory=S_GPS_DIR, file_extensions='.csv',
                           concatenate=False, amt_in_parallel=6)

### 4. Good RFID candidates

In [5]:
S_GOOD_RFID_CANDIDATES = '/Data/Source/RFID/Good_RFID_candidates.csv'

### 5. Map matching report

In [20]:
I_GOOD_RFID_MAP_MATCH = '/Data/Intermediate/MapMatchingReports/GOOD_RFID_ACTIVATIONS.csv'

# 3. Good RFID activations extraction

In [6]:
GOOD_CANDIDATES = [
    ('XE 5598Z', '2020-10-30'),
    ('XE 5612R', '2020-10-30'),
    ('XE 5620S', '2020-10-30'),
    ('XE 5620S', '2020-10-30'),
    ('XE 5628X', '2020-10-30'),
    ('XE 5630M', '2020-10-30'),
    ('XE 5705G', '2020-10-30'),
    ('XE 5598Z', '2020-10-07'),
    ('XE 5612R', '2020-10-07'),
    ('XE 5629X', '2020-10-07'),
    ('XE 5680T', '2020-10-07'),
    ('XE 5598Z', '2020-10-22'),
    ('XE 5612R', '2020-10-22'),
    ('XE 5620S', '2020-10-22'),
    ('XE 5630M', '2020-10-22'),
]

candidates = pd.DataFrame(GOOD_CANDIDATES, columns=['vehicle_id', 'date'])

### 1. Extraction good gps points in map matching report

In [7]:
map_matching_report = \
    map_matching_report.merge(candidates,
                              left_on=['waste_collection_date', 'truck_id'],
                              right_on=['date', 'vehicle_id'])

In [21]:
map_matching_report.to_csv(I_GOOD_RFID_MAP_MATCH, index=False)

### 2. RFID

In [8]:
candidates['vehicle_id'] = candidates['vehicle_id'].str.replace(' ', '-')
rfid = rfid.merge(candidates, on=['vehicle_id', 'date'])

In [9]:
rfid.to_csv(S_GOOD_RFID_CANDIDATES, index=False)

### 3. Good GPS candidates

In [10]:
# source files
files = list(map_matching_report['csv_file'])
S_GPS_FILES = list(map(lambda x,y: x+y, [S_GPS_DIR]*len(files), files))

# Target files
T_GPS_DIR = '/Data/Source/GoodGPSCandidates/'
T_GPS_FILES = list(map(lambda x,y: x+y, [T_GPS_DIR]*len(files), files))

In [11]:
for src, trgt in zip(S_GPS_FILES, T_GPS_FILES):
    pd.read_csv(src).to_csv(trgt, index=False)